# Code Wasteground

This is a notebook containing test code, nothing to see here.

# Setup

In [0]:
SEQ_LEN = 100  # maximum sequence length we want for training
EPOCHS = 48  # number of times we iterate over the full dataset during training
RNN = 'GRU'  # whether we use LSTM or GRU RNN units
UNITS = 2048  # how many units we use
BATCH_SIZE = 64  # no. sequences of SEQ_LEN we train on before updating weights
EMBED_DIM = 256  # vector dimension of word vector embeddings
PRINT = 1000  # how many characters we print during text generation

# 1. Importing Modules and Data

In [0]:
import requests
import re
import numpy as np

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import os

In [0]:
# import Meditations by Marcus Aurelius
response = requests.get('http://classics.mit.edu/Antoninus/meditations.mb.txt')
data = response.text
del response

In [4]:
print(data[:500])

Provided by The Internet Classics Archive.
See bottom for copyright. Available online at
    http://classics.mit.edu//Antoninus/meditations.html

The Meditations
By Marcus Aurelius


Translated by George Long

----------------------------------------------------------------------

BOOK ONE

From my grandfather Verus I learned good morals and the government
of my temper. 

From the reputation and remembrance of my father, modesty and a manly
character. 

From my mother, piety and beneficence, and


# 2. Pre-Processing Data
## 2.1 Cleaning
### 2.1.1 Filter Out
There are several items that we must filter out from Meditations, these are:
<ul>
    <li>everything before the first "---------" line (or everything before and including "Translated by George Long")</li>
    <li>any line that says "BOOK ..."</li>
    <li>any line that is "---------"</li>
    <li>"THE END" and everything after this</li>
</ul>

In [5]:
# remove everything before and including "Translated by George Long"
data = data.split('Translated by George Long')[1]
print(data[:500])



----------------------------------------------------------------------

BOOK ONE

From my grandfather Verus I learned good morals and the government
of my temper. 

From the reputation and remembrance of my father, modesty and a manly
character. 

From my mother, piety and beneficence, and abstinence, not only from
evil deeds, but even from evil thoughts; and further, simplicity in
my way of living, far removed from the habits of the rich.

From my great-grandfather, not to have frequented pub


In [6]:
# remove "----" lines, as "-" is not a useful character we will remove it completely
data = data.replace('-', '')
print(data[:500])





BOOK ONE

From my grandfather Verus I learned good morals and the government
of my temper. 

From the reputation and remembrance of my father, modesty and a manly
character. 

From my mother, piety and beneficence, and abstinence, not only from
evil deeds, but even from evil thoughts; and further, simplicity in
my way of living, far removed from the habits of the rich.

From my greatgrandfather, not to have frequented public schools,
and to have had good teachers at home, and to know that on


In [7]:
# remove "BOOK ..." lines, for this we use regular expressions
data = re.sub('BOOK [A-Z]+\n', '', data)
print(data[:500] + "\n\n...\n\n" + data[-500:])






From my grandfather Verus I learned good morals and the government
of my temper. 

From the reputation and remembrance of my father, modesty and a manly
character. 

From my mother, piety and beneficence, and abstinence, not only from
evil deeds, but even from evil thoughts; and further, simplicity in
my way of living, far removed from the habits of the rich.

From my greatgrandfather, not to have frequented public schools,
and to have had good teachers at home, and to know that on such thi

...

show

THE END



Copyright statement:
The Internet Classics Archive by Daniel C. Stevenson, Web Atomics.
World Wide Web presentation is copyright (C) 19942000, Daniel
C. Stevenson, Web Atomics.
All rights reserved under international and panAmerican copyright
conventions, including the right of reproduction in whole or in part
in any form. Direct permission requests to classics@classics.mit.edu.
Translation of "The Deeds of the Divine Augustus" by Augustus is
copyright (C) Thomas Bushnell

In [8]:
# remove "THE END" and all that follows it
data = data.split("THE END")[0]
print(data[:500] + "\n\n...\n\n" + data[-500:])






From my grandfather Verus I learned good morals and the government
of my temper. 

From the reputation and remembrance of my father, modesty and a manly
character. 

From my mother, piety and beneficence, and abstinence, not only from
evil deeds, but even from evil thoughts; and further, simplicity in
my way of living, far removed from the habits of the rich.

From my greatgrandfather, not to have frequented public schools,
and to have had good teachers at home, and to know that on such thi

...

sh and a tale, or not even a tale.
And let there be present to thy mind also everything of this sort,
how Fabius Catullinus lived in the country, and Lucius Lupus in his
gardens, and Stertinius at Baiae, and Tiberius at Capreae and Velius
Rufus (or Rufus at Velia); and in fine think of the eager pursuit
of anything conjoined with pride; and how worthless everything is
after which men violently strain; and how much more philosophical
it is for a man in the opportunities presented to him to

### 2.1.2 Pull Character-Level Vocab

We do this before splitting the data for character level

In [9]:
vocab_char = sorted(set(data))  # character level vocab
print(f'{len(vocab_char)} unique characters found')

66 unique characters found


In [10]:
print(vocab_char)

['\n', ' ', '!', '"', "'", '(', ')', ',', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


### 2.1.3 Split by Paragraph

In [11]:
# splitting by newline characters
data = data.split('\n\n')
print(data[:10])

['', '', '\nFrom my grandfather Verus I learned good morals and the government\nof my temper. ', 'From the reputation and remembrance of my father, modesty and a manly\ncharacter. ', 'From my mother, piety and beneficence, and abstinence, not only from\nevil deeds, but even from evil thoughts; and further, simplicity in\nmy way of living, far removed from the habits of the rich.', 'From my greatgrandfather, not to have frequented public schools,\nand to have had good teachers at home, and to know that on such things\na man should spend liberally. ', "From my governor, to be neither of the green nor of the blue party\nat the games in the Circus, nor a partizan either of the Parmularius\nor the Scutarius at the gladiators' fights; from him too I learned\nendurance of labour, and to want little, and to work with my own hands,\nand not to meddle with other people's affairs, and not to be ready\nto listen to slander. ", 'From Diognetus, not to busy myself about trifling things, and not\nto 

In [12]:
# remove empty samples
empty = lambda x: x.replace('\s+', '') != ''
data = list(filter(empty, data))
print(data[:10])

['\nFrom my grandfather Verus I learned good morals and the government\nof my temper. ', 'From the reputation and remembrance of my father, modesty and a manly\ncharacter. ', 'From my mother, piety and beneficence, and abstinence, not only from\nevil deeds, but even from evil thoughts; and further, simplicity in\nmy way of living, far removed from the habits of the rich.', 'From my greatgrandfather, not to have frequented public schools,\nand to have had good teachers at home, and to know that on such things\na man should spend liberally. ', "From my governor, to be neither of the green nor of the blue party\nat the games in the Circus, nor a partizan either of the Parmularius\nor the Scutarius at the gladiators' fights; from him too I learned\nendurance of labour, and to want little, and to work with my own hands,\nand not to meddle with other people's affairs, and not to be ready\nto listen to slander. ", 'From Diognetus, not to busy myself about trifling things, and not\nto give cre

In [0]:
# remove final '\n' characters
data = list(map(lambda x: x.replace('\n', ' '), data))

In [14]:
data[:20]

[' From my grandfather Verus I learned good morals and the government of my temper. ',
 'From the reputation and remembrance of my father, modesty and a manly character. ',
 'From my mother, piety and beneficence, and abstinence, not only from evil deeds, but even from evil thoughts; and further, simplicity in my way of living, far removed from the habits of the rich.',
 'From my greatgrandfather, not to have frequented public schools, and to have had good teachers at home, and to know that on such things a man should spend liberally. ',
 "From my governor, to be neither of the green nor of the blue party at the games in the Circus, nor a partizan either of the Parmularius or the Scutarius at the gladiators' fights; from him too I learned endurance of labour, and to want little, and to work with my own hands, and not to meddle with other people's affairs, and not to be ready to listen to slander. ",
 'From Diognetus, not to busy myself about trifling things, and not to give credit to w

In [15]:
print(f"We have {len(data)} stoic lessons from Marcus Aurelius")

We have 507 stoic lessons from Marcus Aurelius


In [0]:
# now join back together in full text
data = '\n'.join(map(lambda x: x.strip(), data))  # we also use map to strip each paragraph

### 2.1.4 Pull Word-Level Vocab

In [17]:
vocab_word = sorted(set(data.split(' ')))  # word level vocab
print(f'{len(vocab_word)} unique words found')

# !!! TODO: add punctuation as it's own word (if we go with word-level)

6454 unique words found


## 2.2 Format Preparation

### 2.2.1 Mapping Tables

In [0]:
char2idx = {u:i for i, u in enumerate(vocab_char)}
idx2char = np.array(vocab_char)

In [0]:
example_idx = np.array([char2idx[c] for c in "Everything we see is a perspective, not the truth."])

In [20]:
print(example_idx)

[16 61 44 57 64 59 47 48 53 46  1 62 44  1 58 44 44  1 48 58  1 40  1 55
 44 57 58 55 44 42 59 48 61 44  7  1 53 54 59  1 59 47 44  1 59 57 60 59
 47  8]


In [0]:
data_idx = np.array([char2idx[c] for c in data])

In [22]:
len(data[0])

1

### 2.2.2 Slicing Data
For the training data, we divide the text into example sequences. Each input sequence is of length <b>seq_length</b>. Each corresponding target sequence will consist of the input sequence shifted one character to the right.

In [23]:
len(data)

241199

In [0]:
# max length sentence we want
seq_length = SEQ_LEN
examples_per_epoch = len(data) // (seq_length+1)

# create training data
char_dataset = tf.data.Dataset.from_tensor_slices(data_idx)

In [0]:
# batch method allows conversion of individual characters to sequences of a desired size
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [0]:
# for each sequence, we duplicate and shift it to form the input and target text
def split_input_output(chunk):
    input_data = chunk[:-1]
    target_data = chunk[1:]
    return input_data, target_data

dataset = sequences.map(split_input_output)

### 2.2.3 Training Batches

Before feeding data to the model, must shuffle the data and pack into batches

In [0]:
# buffer size to shuffle the dataset
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

# 3 Build Model

In [0]:
def build_model(vocab_size, embed_dim, rnn_units, batch_size):
    if RNN == 'GRU':
        model = tf.keras.Sequential([
            tf.keras.layers.Embedding(vocab_size, embed_dim,
                                    batch_input_shape=[batch_size, None]),
            tf.keras.layers.GRU(rnn_units, return_sequences=True,
                                stateful=True,
                                recurrent_initializer='glorot_uniform'),
            tf.keras.layers.Dense(vocab_size)
        ])
    elif RNN == 'LSTM':
        model = tf.keras.Sequential([
            tf.keras.layers.Embedding(vocab_size, embed_dim,
                                    batch_input_shape=[batch_size, None]),
            tf.keras.layers.LSTM(rnn_units, return_sequences=True,
                                 stateful=True,
                                 recurrent_initializer='glorot_uniform'),
            tf.keras.layers.Dense(vocab_size)
        ])
    return model

In [0]:
model = build_model(
    vocab_size=len(vocab_char),
    embed_dim=EMBED_DIM,
    rnn_units=UNITS,
    batch_size=BATCH_SIZE)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16896     
_________________________________________________________________
gru (GRU)                    (64, None, 2048)          14168064  
_________________________________________________________________
dense (Dense)                (64, None, 66)            135234    
Total params: 14,320,194
Trainable params: 14,320,194
Non-trainable params: 0
_________________________________________________________________


## Note:

For some reason, the same model built another way causes an error when generating text. I have asked why on the following Stackoverflow page:


The alternative code causing the error is:



```
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embed_dim,
                                        batch_input_shape=[BATCH_SIZE, None]))
if RNN == 'GRU':
    model.add(tf.keras.layers.GRU(UNITS, return_sequences=True,
                                  stateful=True,
                                  recurrent_initializer='glorot_uniform'))
else:
    model.add(tf.keras.layers.LSTM(UNITS, return_sequences=True,
                                   stateful=True,
                                   recurrent_initializer='glorot_uniform'))
model.add(tf.keras.layers.Dense(vocab_size))
```



# 4. Training

In [0]:
# define the loss function
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [0]:
# compile the model
model.compile(optimizer='adam', loss=loss)

In [0]:
# configure checkpoints
checkpoint_dir = './training_checkpoints'
# name of files
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [0]:
# execute training
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 37 steps
Epoch 1/48
37/37 [==============================] - 724s 20s/step - loss: 3.6687
Epoch 2/48
37/37 [==============================] - 715s 19s/step - loss: 2.4088
Epoch 3/48
37/37 [==============================] - 720s 19s/step - loss: 2.2125
Epoch 4/48
37/37 [==============================] - 723s 20s/step - loss: 2.0993
Epoch 5/48
37/37 [==============================] - 726s 20s/step - loss: 1.9945
Epoch 6/48
37/37 [==============================] - 714s 19s/step - loss: 1.8874
Epoch 7/48
37/37 [==============================] - 722s 20s/step - loss: 1.7863
Epoch 8/48
37/37 [==============================] - 716s 19s/step - loss: 1.6946
Epoch 9/48
37/37 [==============================] - 720s 19s/step - loss: 1.6105
Epoch 10/48
37/37 [==============================] - 723s 20s/step - loss: 1.5355
Epoch 11/48
37/37 [==============================] - 714s 19s/step - loss: 1.4697
Epoch 12/48
37/37 [==============================] - 716s 19s/step - loss: 1.4141
Epoch 

# 5. Generate Text

## 5.1 Restore Final Checkpoint
Because of the way the RNN state is passed from timestep to timestep the model only accepts a fixed batch size once built.

To run the model with another batch_size we rebuild to model and restore weights from the checkpoint

<b>maybe this can be done in Tensorflow.js?</b>

In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

In [0]:
model = build_model(len(vocab_char), EMBED_DIM, UNITS, 1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

In [0]:
# defining text generation function
def generate_text(model, start_string, length=1000, temp=1.0):
    # low temp = more predictable text, high temp = more surprising text

    # converting start string to numbers (vectorisation)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # initialise empty string to store results
    text = []

    # batch size is 1
    model.reset_states()
    for i in range(length):
        predictions = model(input_eval)
        # remove batch dimension
        predictions = tf.squeeze(predictions, 0)

        # use categorical distribution to predict character returned by model
        predictions = predictions / temp
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # we pass the predicted character as the next input to the model
        # along with previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        # append predicted text
        text.append(idx2char[predicted_id])

    return (start_string + ''.join(text))

In [0]:
print(generate_text(model, start_string=u'From ', length=PRINT))

In [0]:
history

# 12 iterations
## GRU

<b>From</b> any placest is so.

Thou art dods thee from the deadgh of impuety by habour of mecrating, and oter coused by doing with gaves indifected, if thou hass serdect to him at a think. The way only not ando, nothing is compousted the that happoness; and de to be exturn and I he another suchivisicis inforaition with its Iw all vellected with does at all things, how sefull: and kill not the beact?

That this a soun of the saie past, but lothing else is gorally that they relamas; but thy chatical and injiencque its shall goderatis, or one oppens; and how the raying phatiet of thy negerce, and a stead. And us our bears prriaced from however suagled with the places with many by other by social interest findly op not deity it is of a way a man it has subce into says of a shall call the thing in thems experacled which good feruble them, but Alsose and blamon, and welse hassides, fruedome power life, them wouldsh tranfining procurnation.

There would harm he whole could a portur and a reculsm for this i

# 24 iterations
## GRU
### 1024 Units

<b>From</b> the intelligent collision, and to his consistlation or unfustiles respect to the rapidity, caln: thou, wilt last to live prevent through a good man, us afor on to that eit when has myser his present quuriely? For own nects fortus I have completed, to discover that thou mayest also does no harm to this.

Not turn to thee consist of nature is a thing, for it truly see that thou thyself still firl into them. Altogether to come at thing is a good forte: and if thou to me the cause; but in the sturce that my cherich are honextife, it would thou not seek flattering of thy nature that all the things which are simple, the strangerswat man's intelligence and yet died every men keep myself, and so affectionate, and as then cannothe other social sopical or God.

The reason resolutes fer investiguted without sufficient to everything more, boccolds to everything which is oblivion, and to evil to my excare imposed to me, thou are the appettable, thou wilt recorce so let not too. In the simple which ha

## LSTM

<b>From</b> understand whom they dissolution died. Further, neither is a rocernal counterest, come for thee are formed by nature; for they are when it both in thy power. Well, sump things with been, he should sespord, but in so is shown; whe must discovering thy benevolent des with see; but that it some within without tre with that with which thou hast seen Another in my opinions. And thou hadst separated from life.

Tourght of ndire, so many after asidged his portion, or down and of that which caul and ready eythy of one according to existence out of what is not caul say or Paway of or altog, and that it will be either affairates, and hon that thou hast been a knowledoe, and in can, Be in a handreit for something else.

Phar has been constituted, firellesturiou was its own act chance of him, and remedness of which a man beings what happens and in sumpersh, throesonu, will be obssiedly when thou arother, shall care after different things; and he choose there of whom universal, death, or is intellige

# 48 iterations
## GRU
### 1024 Units

<b>From</b> of those who live with us and speak; and that we ought not, like children.

The Lace after the preshn and all to the state, do not to be such as the whole of thy life. For as soon as I have freedom of which everything harmbeing and deprived of the power of bistence? And if they get as doing at the same time having a reference to the intelficable and profitable! And yet have no rouses and us unter out of the Ephesians there was this precept, thou mightest remember that thy attempt was right to vexattroperate for these purpose. And this farout inquiry, moves strain those who have d soul, andore erect, not be deprived of the soul of man, and to the soul of God.

All things are the condidion of the universe to be none by any exalted. For one evolence is it with reference to the thouard, thou wilt see what I say, if thou only desire to be a good man in earthy turns be the imagination. Stop the pulling of the strings. Confine thyself to the things which are presented to it, st parts, whicerdon

### 2048 Units

...

# 64 Iterations

## 1024 Units
### SEQ_LEN = 100

From many together with more quiet or more freedom from trouble does no harm to the elements themselves in the soul, if it be consistent with social interests, to the Universe will be example and blate with my own hands, and not to be seen by the gods neither dissatisfied with any solid body which stands, and be to consider that the men of after time which are in out to the earse (reason; and the activities of things are in consequence of change.

Time is useful to thee and words it, and do not for thishout turning back: but if thou canst shom which a vert excuse d good quirity are on om roo.
kin, and to look for the fig in winter is, which has changed, and that it without deathred any at back; and whet have designed.

Does another do me wrong? Let him look to this only in all that he does, whether he is doing what is just, and so secure the saying of Hy nature, and everything else is of the man as if wo far as soon as thou findest many other differences at the games in which thou hast abvert

### SEQ_LEN = 300

From I mout dyems; but let this prescribed this kind andorgance is a polity which is not life, retarn to thyself, motever, they to care atail for the poor bless society. Now mutation of epicials accuptom that no man would not have the country of the food free from all emptying sufficient confly than to be hindered. Thou wilt see how a fightence just see is an emply things both flate and life, if immediately passed the opinion? And accordingly men soone then being hadden's activity, and towards mutual compair in my opinion, thou also when the whill in his deserts, a right the desires of are, it being contentment with him who have right as the mottern thres himself to other things.

To power is more that his life indoed instruption at all to those who brongs the bad man do not being reverence to oureche is an object. So far as he should pragree some other naburity have been mistered? We hust thee that these persons so die, it last bo it. And he has a mea human being, having with mid and felici


# Hypothesis
Increasing SEQ_LEN increases the continuity of language, but requires more iterations in order to understand smaller sections of language as well as decreased SEQ_LEN values.

> Try SEQ_LEN=300 with EPOCHS=128

# 128 Iterations
### SEQ_LEN = 300

From Sheersally so faultain by it.

Let him look to it. But I will be mild and benevolent towards all things which man for those hourself, I shall bear in mind that he is compelled to do so.

Remember that as it is a shame to be surprised if the figtree produces figs, so it is to be surprised, if a man has a fever, or if it were the last, laying aside all carelessness and passionate aversion from the commands of reason, well then, if it is useful to thee as a rational being, keep to it; but if it is only useful to the universal nature. And consider that as the heasons of thy life bring, such also is dissolution. This, then, is consistency wat full in life bousest for a hundred years or threaf and the constituted for one cooperation. And the perception dull, and the composition of the whole body subject to putrefaction, and the soul a whirl, and fortune hard to divine, and fame aw being valued by them; nor in its nothing which it does not allow the fiety and free and modesty and with men, they

# 254 Iterations

## SEQ_LEN = 300

From my grandfather Verus I learned good morals and the government of my temper.

From the reputation and remembrance of my father, modesty and a manly character.

From my mother, piety and beneficence, and abstinence, not only from evil deeds, but even from evill things will perish into honour, which they seemed to desire, without putting them off with hope of my doing it some time after, because they were then still young; that I knew Apollonius, Rustice, and to hish they are more such effort as this. But thou must exent to be accomplished on the illness of all things, and he is Law, and assigns to every man what is fit.

He who has a vehement desire for posthumous fame does not consider that every one of those who remember him will himself also die very soon; then again also they who have succeeded them, until the whole remembrance shall have been extinguished as it is transmitted through men who fears or is grieved opportunity offers, thou fend towards them, for by nature they are friends. And the gods too aid them in all ways, by dreams, by signs, towards the attainment of those things on which they set a value.

The periodic movements of the universe are the same, up and down from age to age. And either the universal intelligence puts itself, I man directs its own movement are governed by no rational principle. If this is remembered it will make thee more tranquil in many things are suitable to us, arm to the deity, so far as I am A city as in thy power? Take away these opinions then, and resolve to despise them, and to do good to men, and to practise tolerance to an act; and that which is an obstacle on the road helps us on this road.

Reverence that which is diseased.

Shortlived are both the praiser and the praised, and the rememberer and the dead word of what thou hast known, one after another. One man after burying another has been laid out dead, and another buries him: and all that is allotted, as coming from thence, wherever it is, from whence he himself came; and, finally, waiting for death with a cheerful men or irrational? Souls of rational men. Sound or unsound? Sound. Why then do you not seek for them? Because thou hast seen Xany indeed have not been remembered even for a short time, and others have become the heroes of fables, and again others have disappeared even from fables. Remember this then, that this little compound, thyself, must either be dissolved, or thy duty to comf to thyself: For thou art a part. But that his own nature. And it is in his power to do according to his own nature. And it is in his power everywhere. Now, it is not given to a cylinder towards the venything of this knether thing to the peculiarly in old, are now in a manner antiquated, Camillus, Caeso, Volesus, Leonnatus, and a little after also Shing which happens is a separation of ourselves from nature, in some part of which the natures of all other things are contained. In the next place, the soul does violence to itself when it turns away from any man, or eve

# Note
I imagine this is overfitted, it is essentially copy and paste from the original text unfortunately, so maybe shorter SEQ_LEN and less iterations. Loss reached ~0.05.

# 50 SEQ_LEN tests
## 64 Iters
### LSTM

...

### GRU

...

In [0]:
``